## Sensor input simulation

In [ ]:
import paho.mqtt.client as mqtt
import time
from random import random
import json

### Setup of mqtt client

In [ ]:
# Definition of callbacks that are automatically called
# on certain events (connection, message sent, message received, etc.)

def on_connect(client, userdata, flags, rc):
    print("Connected to message broker with result code " + str(rc))

def on_message(client, userdata, msg):
    print(msg.topic + " " + str(msg.payload))
    
def on_disconnect(client, userdata, rc):
    print("Disconnected from message broker with result code " + str(rc))
    client.loop_stop()
    
def on_publish(client, userdata, mid):
    print("Message successfully sent to broker")

In [ ]:
# Initialization of client and linking to our callbacks
agent_edge = mqtt.Client(client_id = "AgentEdge")
agent_edge.on_connect = on_connect
agent_edge.on_message = on_message
agent_edge.on_disconnect = on_disconnect
agent_edge.on_publish = on_publish

### Setup of sensor objects

In [ ]:
# Definition of two simulated sensors. Only placeholders for the real sensors.
class SensorTempHum(object):
    
    def __init__(self):
        pass
    
    def get_temperature(self):
        return (random() * 60) - 20
    
    def get_humidity(self):
        return random()

In [ ]:
class SensorLight(object):
    
    def __init__(self):
        pass
    
    def get_light(self):
        return random() * 10000

### JSON setup

In [ ]:
# Use dictionaries for JSON messages, since they easily convert to JSON
# All values except those under "values" remain the same throughout the run

msg_temphum = {"deviceId": "SensorDemo01",
               "requestId": "SensorDemo01_getTempHum",
               "serviceName": "sensorService",
               "actionName": "getTempHum",
               "type": "EVENT",
               "level": "INFO",
               "values": 
                   {"temperature": "N/A",
                    "humidity": "N/A"
                   }
              }

msg_light = {"deviceId": "SensorDemo01",
             "requestId": "SensorDemo01_getLight",
             "serviceName": "sensorService",
             "actionName": "getLight",
             "type": "EVENT",
             "level": "INFO",
             "values": 
                 {"light": "N/A"
                 }
              }

### Main loop

In [ ]:
hostname = "localhost" # Replace with IP adress for an externally hosted broker
interval = 5 # Number of seconds between each time data is sent

# Connect to the host
agent_edge.connect(hostname)

# Start loop to ensure that the connection is maintained
agent_edge.loop_start() 

# Initialize connections to the sensors
sensor_th = SensorTempHum()
sensor_l = SensorLight()

# Main loop
for i in range(5):
    # Get data from sensors and put them in the message dictionaries
    msg_temphum["values"]["temperature"] = "{:.2f}".format(sensor_th.get_temperature())
    msg_temphum["values"]["humidity"] = "{:.3f}".format(sensor_th.get_humidity())
    msg_light["values"]["light"] = "{:.2f}".format(sensor_l.get_light())
    
    # Publish the messages to the broker using the topic "home" (topic is mandatory to include)
    agent_edge.publish("home", json.dumps(msg_temphum))
    agent_edge.publish("home", json.dumps(msg_light))
    
    # Wait for X seconds until the next message is sent
    time.sleep(interval)
    
# Disconnect and shut down
print("Shutting down client and disconnecting.")
agent_edge.disconnect()